# **Examen final de Desarrollo de Aplicaciones para la Visualización de Datos**

Nombre: Antonio

Apellidos: Pardo de Santayana Navarro

Tiempo: *2 horas y 30 minutos*

## **Contexto del ejercicio**

Un banco portugues desea entender en más detalle las campañas de marketing directo que ha realizado en los últimos meses a más de 40 mil clientes. Las campañas de marketing se basaron en llamadas telefónicas. Muchas veces era necesario más de un contacto con un mismo cliente, para saber si el producto (depósito bancario) sería ('sí') o no ('no') contratado.

El objetivo del análisis es buscar patrones para entender mejor que tipo de perfil tienen los clientes que han contratado el depósito para buscar en su base de datos otros clientes parecidos para aumentar la respuesta y el ROI de futuras campañas de marketing directo intentando vender el mismo depósito. Por lo tanto, se pide:

1.  Realizar un análisis descriptivo de los datos con al menos, 6 visualizaciones diferentes. (3 Puntos) (*)

2.  Montar un dashboard con al menos, 4 visualizaciones diferentes, que incluyan 2 componentes interactivas y 1 callback (5 Puntos) (*)

3. Concluir todo este análisis haciendo recomendaciones para la mejora de futuras campañas de contacto directo a partir de los resultados obtenidos de los análisis realizados con los datos. (2 Puntos)

Para realizar este análisis se provee de un juego de datos con las siguientes variables:

- age	- Edad (numérica)
- job - Ocupación (categórica)
- marital - Estado civil (categórica)
- education - Nivel educativo (categórica)
- default - ¿Tiene algún prestamo en default? (binaria)
- balance - Balance medio anual en euros (numérica)
- housing - ¿Tiene una hipoteca? (binaria)
- loan -  ¿Tiene un prestamo personal? (binaria)
- contact - Tipo de contacto (categórica)
- day_of_week - Último día de la semana que fue contactado (fecha)
- month - Último mes que fue contactado (fecha)
- duration - Duración de la última vez que fue contactado en segundos (entera)
- campaign - Número de veces que fue contactado (numérica)
- pdays - Número de días que pasaron después de que el cliente fue contactado por última vez desde una campaña anterior (numérica; -1 significa que el cliente no fue contactado previamente)
- previous - Número de contactos realizados durante esta campaña y para este cliente (numérica, incluye el último contacto)
- poutcome - Resultado de la campaña de marketing anterior (categórica; 'failure','nonexistent','success')
- y - El cliente ha contratado el depósito (binaria, yes, no)

Recuerda, si tuvieras que programar una función, comenta los argumentos de entrada y salida. **Explica el orden que estás siguiendo a la hora de elegir las visualizaciones y comenta las conclusiones que vas sacando.**


**(*) IMPORTANTE**: Puedes elegir realizar un modelo de clasificación y realizar visualizaciones en torno a ese modelo en los primeros dos apartados. Esta parte no es obligatoria. El objetivo de la clasificación sería predecir si el cliente se suscribirá a un depósito bancario (variable y).








### **Librerías necesarias**

In [76]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go

### **Análisis descriptivos**

In [132]:
df = pd.read_csv('bank-full.csv', sep=';').dropna()
df_clients = df[df['y'] == 'yes']
df_rejections = df[df['y'] == 'no']
print(df.head())
print(df_clients.head())
print(df_clients.shape)

   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1         0  unknown  no  
     age         job  marital  education default  balance housing loan  \
83    59      admin.  married  second

### Parte 1

Comenzamos intentando conocer mejor al cliente que contrata nuestro servicio, para ello vamos a ver como se comporta nuestra clientela con dos modelos de ML.

In [133]:
# model used - random forest to find importance
def fit_random_forest(df):
    '''Arguments:
    df [dataframe] : dataframe upon which to make the classifier
    Returns:
    importances [dataframe] : importances of each variable'''
    #Nos quedamos solo con algunas variables binarias que nos interesen
    X = pd.get_dummies(df.drop(['contact', 'month', 'day', 'poutcome', 'pdays', 'duration', 'age', 'balance', 'previous', 'campaign', 'y'], axis=1))
    y = df['y']
    # No separamos en train y test ya que no es tan importante el rigor del modelo, solo queremos las importancias
    rf = RandomForestClassifier().fit(X, y)
    return  pd.DataFrame({
        'Var': list(X.columns),
        'Importance':list(rf.feature_importances_)
    })

def fit_logistic_regression(df):
    '''Arguments:
    df [dataframe] : dataframe upon which to make the classifier
    Returns:
    coefficients [dataframe] : coefficient of each variable'''
    X = df[['age', 'balance', 'duration']]
    y = df['y']
    ss = StandardScaler()
    X_scaled = ss.fit_transform(X)
    lr = LogisticRegression().fit(X_scaled, y)
    #ignoramos el intercept
    return pd.DataFrame({
        'Var': list(X.columns),
        'Coefficient': list(lr.coef_[0])
    })

def fit_lr_contact(df):
    '''Arguments:
    df [dataframe] : dataframe upon which to make the classifier
    Returns:
    coefficients [dataframe] : importances of each variable'''
    #Nos quedamos solo con algunas variables binarias que nos interesen
    X = pd.get_dummies(df[['contact', 'month']])
    y = df['y']
    # No separamos en train y test ya que no es tan importante el rigor del modelo, solo queremos las importancias
    ss = StandardScaler()
    X_scaled = ss.fit_transform(X)
    lr = LogisticRegression().fit(X_scaled, y)
    return pd.DataFrame({
        'Var': list(X.columns),
        'Coefficient': list(lr.coef_[0])
    })

In [134]:
# figure 1: age distribution of clients. show destroys fig object so we save it in another
fig_age = px.histogram(df_clients, x='age', title='Age Distribution of Clients')
fig = px.histogram(df_clients, x='age', title='Age Distribution of Clients')
fig.show()

In [135]:
## figure 2: ocupation and education of our clients
df_job = df.groupby(['job'])['job'].count().reset_index(name='count')
fig_job = px.pie(df_job, values='count', names='job', title='Ocupation')
fig = px.pie(df_job, values='count', names='job', title='Ocupation')
fig.show()


In [136]:
### Figure 3: Variable importances according to our model
importances = fit_random_forest(df)
importances.sort_values('Importance', ascending=False, inplace=True)
fig = px.bar(importances.head(10), y='Var', x='Importance', orientation='h')
fig.show()



In [137]:
# figure 4: ahora queremos ver como se relaciona el balance de nuestros clientes según su edad
fig_balance = px.scatter(df_clients, x='age', y='balance')
fig = px.scatter(df_clients, x='age', y='balance')
fig.show()

In [138]:
# figure 5: box plot of contacted for no clients and yes clients to see whether calling many times helps

fig = go.Figure()
fig.add_trace(go.Box(y=df_clients['campaign'], boxpoints=False))
fig.add_trace(go.Box(y=df_rejections['campaign'], boxpoints=False))

fig.show()

In [139]:
# figure 6:
coefs = fit_logistic_regression(df)
fig = px.bar(coefs, x='Var', y='Coefficient')
fig.show()

In [140]:
# figure 7 know when and how to contact
importances_contact = fit_lr_contact(df)
importances_contact.sort_values('Coefficient', ascending=False, inplace=True)
fig = px.bar(importances_contact, x='Var', y='Coefficient')
fig.show()

###

### Parte 2: Dashboard

The most important data are the results of our regressions, so that will be included in the dashboard, the final

In [145]:
from dash import dcc, html, Input, Output
import dash
app = dash.Dash()
plots = ['Age Distribution', 'Jobs Pie chart', 'Balance by Age']
app.layout = html.Div([
    html.H1('Marketing Campaign Recommendations'),
    html.Div([
    #topleft
    html.Div([
        dcc.Graph(figure=go.Bar(y=importances['Var'], x=importances['Importance'], orientation='h', name='Who is our Client?'))
    ], style={'display': 'inline-block', 'width' : '60%', 'verticalAlign': 'top'}),
    #topright
    html.Div([
        dcc.Graph(figure=go.Bar(y=importances_contact['Var'], x=importances_contact['Coefficient'], name='How to Contact Prospects?'))
    ], style={'display': 'inline-block', 'width': '40%', 'verticalAlign': 'top'}),

    #botleft
    html.Div([
        dcc.Graph(figure=go.Bar(y=coefs['Coefficient'], x=['Var'], name='Age, Balance, or Duration?'))
    ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top'}),

    #botright
    html.Div([
        dcc.Dropdown(id='dropdown-bot-right', options=[{'label': var, 'value': var} for var in plots],
                    value='Unemployment', clearable=False, style={'width': '200px', 'display': 'inline-block'}),
        dcc.Graph(id='bot-right-plot')
    ], style={'display': 'inline-block', 'width': '70%', 'verticalAlign': 'top'})
    ])
])

@app.callback(
    Output('bot-right-plot', 'figure'),
    Input('dropdown-bot-right', 'value')
)
def update_fig_botright(value):
    # figures defined above
    if value == 'Age Distribution':
        return fig_age
    if value == 'Jobs Pie chart':
        return fig_job
    if value == 'Balance by Age':
        return fig_balance

# app.run()

### Parte 3: Conclusión y Recomendaciones

Viendo la información sacada no sólo del dashboard, si no también de las primeras visualizaciones, se pueden sacar las siguientes conclusiones acerca de nuestros clientes:

Primero, tienden a tener una edad dispersa, pero mayoritariamente entre los 30 y 40, por lo que recomendamos acotar las campañas de publicidad a este conjunto demográfico.

Segundo, es muy importante para que nuestro cliente contrate nuestro producto que tenga una casa, siendo mucho más probable que nos contrate si la tiene. Además, también es importante tener en cuenta que estudiantes y jubilados tienen menos probabilidades de contratarnos, acotando aún más nuestro público objetivo, también juega un papel importante que nuestro cliente tenga un préstamo personal contratado. Todo esto apunta a padres y madres de familia relativamente jóvenes y de hecho, también ayuda que estén casados.

Además, la hora de contratar el servicio, la duración de la llamada es lo más importante, por lo que se recomienda a los que llaman retener lo máximo posible al cliente. También es más importante que el balance del cliente sea alto, por lo que se recomienda priorizar a clientes con más dinero en la cuenta.

Otra recomendación importante se encuentra en cuando y como contactar con el cliente: es mejor contactar al móvil que al fijo, por lo que se podría centrar la campaña en números móviles (aunque este hallazgo puede estar correlado con los anteriores) y, sobre todo, contactar en los meses de marzo, junio, septiembre y octubre, ya que son los que mejores resultados dan. Se da el caso opuesto para julio y agosto.

En conclusión, para mejorar el ROI de nuestra campaña recomendamos contactar a personas de entre 30 y 40 años, casados, con préstamos personales y con un balance lo más alto posible, y se recomienda hacerlo principalmente en septiembre y octubre.